In [5]:
import pandas as pd
import numpy as np
import json
from scipy import stats
import tqdm

# Funzioni

In [12]:
def nclusters(data, threshold):
    data = [float(el) for el in data]
    data = sorted(data)
    start = data[0]
    max_val = start + threshold
    c = (start, max_val)
    cluster = dict()
    for i in data:
        if i <= max_val:
            if c in cluster.keys():
                cluster[c] += 1
            else:
                cluster[c] = 1
        else:
            max_val = i + threshold
            c = (i, max_val)
            cluster[c] = 1
    #ora ho il dizionario con i cluster di una run
    C_num = len(data)**2
    C_den = 0
    for k in cluster.keys():
        C_den += cluster[k]*cluster[k]
    C = C_num / C_den
    return C

def entropy(opinions, n, nbins):
    bincounts, bin_edges = np.histogram(opinions, bins = np.linspace(0, 1, nbins))
    probabilities = bincounts/n
    entr = stats.entropy(probabilities)
    return entr

def nodeperc_withininterval(opinions, center, width):
    sortedops = sorted(opinions)
    sortedopsarr = np.array(sortedops)
    withinrange = np.where(np.logical_and(sortedopsarr>=(center-width), sortedopsarr<=(center+width)))
    withinrange = withinrange[0]
    perc = len(withinrange) / len(sortedops)
    return perc

def average_opinions(opinions):
    a = np.array(opinions)
    return np.average(a)

def std_opinions(opinions):
    a = np.array(opinions)
    return np.std(a)

def median_opinions(opinions):
    a = np.array(opinions)
    return np.median(a)
maxentr10 = entropy(np.random.uniform(0.0, 1.0, 250), 250, 11)

In [13]:
model = "rewiring"

In [14]:
finalfile = f"{model} results.csv"

In [15]:
#create one aggregate file
import os
aggrfile = open(f"aggregate/{finalfile}", "w+")
header = "model,graph,n,p,pr,eps,gam,max_it\n"
aggrfile.write(header)
aggrfile.close()
aggrfile = open(f"aggregate/{finalfile}", "a+")
for graph in ['er', 'ba']:
    if graph == "er":
        p = 0.1
    else:
        p=5
    for pr in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
        for eps in [0.2, 0.3, 0.4]:
            for gam in [0.0, 0.4, 0.8, 1.2, 1.6]:
                name = f"{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000"
                s = f"{model},{graph},250,{p},{pr},{eps},{gam},100000\n"
                aggrfile.write(s)
aggrfile.close()

In [16]:
results = pd.read_csv(f"aggregate/{finalfile}")
results

,model,graph,n,p,pr,eps,gam,max_it
0,rewiring,er,250,0.1,0.0,0.2,0.0,100000
1,rewiring,er,250,0.1,0.0,0.2,0.4,100000
2,rewiring,er,250,0.1,0.0,0.2,0.8,100000
3,rewiring,er,250,0.1,0.0,0.2,1.2,100000
4,rewiring,er,250,0.1,0.0,0.2,1.6,100000
...,...,...,...,...,...,...,...,...
175,rewiring,ba,250,5.0,0.5,0.4,0.0,100000
176,rewiring,ba,250,5.0,0.5,0.4,0.4,100000
177,rewiring,ba,250,5.0,0.5,0.4,0.8,100000
178,rewiring,ba,250,5.0,0.5,0.4,1.2,100000


In [21]:
avg = []
std = []
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_iterations {name}.json')
    data = json.load(jsonfile)
    finalitarr = []
    for nr in data.keys():
        finalitarr.append(data[nr])
    finalitarr = np.array(finalitarr)
    avg.append(np.average(finalitarr))
    std.append(np.std(finalitarr))
results["avg_nit"] = avg
results["std_nit"] = std
results["nruns"] = nr+1

doing rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0

TypeError: can only concatenate str (not "int") to str

In [ ]:
avg01 = []
std01 = []
avg001=[]
std001=[]
avg00001=[]
std00001=[]
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    ncarray01 = []
    ncarray001 = []
    ncarray00001 = []
    for nr in data.keys():
        finalops = list(data[nr])
        nc01 = nclusters(finalops, 0.1)
        nc001 = nclusters(finalops, 0.01)
        nc00001 = nclusters(finalops, 0.0001)
        ncarray01.append(nc01)
        ncarray001.append(nc001)
        ncarray00001.append(nc00001)
    ncarray01 = np.array(ncarray01)
    ncarray001 = np.array(ncarray001)
    ncarray00001 = np.array(ncarray00001)
    avg01.append(np.average(ncarray01))
    std01.append(np.std(ncarray01))
    avg001.append(np.average(ncarray001))
    std001.append(np.std(ncarray001))
    avg00001.append(np.average(ncarray00001))
    std00001.append(np.std(ncarray00001))
results["avg_ncluster_01"] = avg01
results["std_ncluster_01"] = std01
results["avg_ncluster_001"] = avg001
results["std_ncluster_001"] = std001
results["avg_ncluster_00001"] = avg00001
results["std_ncluster_00001"] = std00001

doing rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0

In [ ]:
results.to_csv(f"aggregate/{finalfile}")

In [22]:
results = pd.read_csv(f"aggregate/{finalfile}", index_col=[0])
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,nruns,avg_ncluster_01,std_ncluster_01,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001
0,rewiring,er,250,0.1,0.0,0.2,0.0,100000,134.933333,143.666032,29,1.941154,0.142429,1.941780,0.142840,1.941840,0.142899
1,rewiring,er,250,0.1,0.0,0.2,0.4,100000,79.233333,17.406289,29,2.088369,0.344935,2.088403,0.344906,2.088419,0.344900
2,rewiring,er,250,0.1,0.0,0.2,0.8,100000,675.100000,209.840312,29,2.181309,0.420929,2.181451,0.420931,2.181451,0.420931
3,rewiring,er,250,0.1,0.0,0.2,1.2,100000,24207.433333,7222.889379,29,2.468978,0.439708,2.505992,0.458518,2.506278,0.458442
4,rewiring,er,250,0.1,0.0,0.2,1.6,100000,84369.300000,28789.911101,29,8.061771,0.485696,32.917964,3.731579,61.510262,11.134424


In [23]:
averageops = []
averagestdops = []
averagemedianops = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    avgarray = []
    stdarray = []
    medarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        avg = average_opinions(finalops)
        med = median_opinions(finalops)
        std = std_opinions(finalops)
        avgarray.append(avg)
        medarray.append(med)
        stdarray.append(std)
    avgarray = np.array(avgarray)
    medarray = np.array(medarray)
    stdarray = np.array(stdarray)
    averageops.append(np.average(avgarray))
    averagestdops.append(np.average(stdarray))
    averagemedianops.append(np.average(medarray))
results["avg_mean_opinion"] = averageops
results["avg_std_opinions"] = averagestdops
results["avg_median_opinion"] = averagemedianops

doing rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0

In [24]:
results.to_csv(f"aggregate/{finalfile}")

In [25]:
results = pd.read_csv(f"aggregate/{finalfile}", index_col=[0])
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,nruns,avg_ncluster_01,std_ncluster_01,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001,avg_mean_opinion,avg_std_opinions,avg_median_opinion
0,rewiring,er,250,0.1,0.0,0.2,0.0,100000,134.933333,143.666032,29,1.941154,0.142429,1.941780,0.142840,1.941840,0.142899,0.502615,0.230020,0.510947
1,rewiring,er,250,0.1,0.0,0.2,0.4,100000,79.233333,17.406289,29,2.088369,0.344935,2.088403,0.344906,2.088419,0.344900,0.499437,0.235517,0.538134
2,rewiring,er,250,0.1,0.0,0.2,0.8,100000,675.100000,209.840312,29,2.181309,0.420929,2.181451,0.420931,2.181451,0.420931,0.502255,0.238240,0.581333
3,rewiring,er,250,0.1,0.0,0.2,1.2,100000,24207.433333,7222.889379,29,2.468978,0.439708,2.505992,0.458518,2.506278,0.458442,0.502371,0.249277,0.488914
4,rewiring,er,250,0.1,0.0,0.2,1.6,100000,84369.300000,28789.911101,29,8.061771,0.485696,32.917964,3.731579,61.510262,11.134424,0.496690,0.279067,0.487743


In [17]:
values_avg = []
values_std = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    entrarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        entr = entropy(finalops, 100, 11)
        entrarray.append(entr)
    entrarray = np.array(entrarray)
    values_avg.append(np.average(entrarray))
    values_std.append(np.std(entrarray))

results["10B_avg_entr"] = values_avg
results["10B_std_entr"] = values_std

results['10B_avg_norm_entr'] = results['10B_avg_entr'].apply(lambda x: x/maxentr)

doing rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing rewiring er0.1 pr0.1 e0.2 g0.8 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.2 mi100000
doing rewiring er0.1 pr0.1 e0.2 g1.6 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0.0 mi100000
doing rewiring er0.1 pr0.1 e0.3 g0

In [18]:
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_ncluster_01,std_ncluster_01,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001,avg_mean_opinion,avg_std_opinions,avg_median_opinion,10B_avg_entr,10B_std_entr,10B_avg_norm_entr
0,rewiring,er,250,0.1,0.0,0.2,0.0,100000,2.170596,0.476778,2.170636,0.476767,2.202778,0.512769,0.493026,0.235189,0.443834,0.789040,0.192599,0.344689
1,rewiring,er,250,0.1,0.0,0.2,0.4,100000,2.413687,0.525845,2.413696,0.525832,2.542178,0.619739,0.503692,0.240937,0.455000,0.886833,0.201170,0.387410
2,rewiring,er,250,0.1,0.0,0.2,0.8,100000,2.228691,0.386793,2.229085,0.386658,2.364724,0.382978,0.506752,0.237929,0.496806,0.830325,0.156175,0.362724
3,rewiring,er,250,0.1,0.0,0.2,1.2,100000,2.579699,0.445250,2.645225,0.471204,2.742667,0.555480,0.500606,0.249831,0.546255,1.008177,0.179297,0.440418
4,rewiring,er,250,0.1,0.0,0.2,1.6,100000,7.854710,0.402084,32.456240,5.061050,64.245012,10.079154,0.492880,0.277877,0.486494,2.220175,0.038018,0.969876


In [19]:
results.to_csv("aggregate/{}".format(finalfile))
results = pd.read_csv("aggregate/{}".format(finalfile), index_col=[0])

In [20]:
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_ncluster_01,std_ncluster_01,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001,avg_mean_opinion,avg_std_opinions,avg_median_opinion,10B_avg_entr,10B_std_entr,10B_avg_norm_entr
0,rewiring,er,250,0.1,0.0,0.2,0.0,100000,2.170596,0.476778,2.170636,0.476767,2.202778,0.512769,0.493026,0.235189,0.443834,0.789040,0.192599,0.344689
1,rewiring,er,250,0.1,0.0,0.2,0.4,100000,2.413687,0.525845,2.413696,0.525832,2.542178,0.619739,0.503692,0.240937,0.455000,0.886833,0.201170,0.387410
2,rewiring,er,250,0.1,0.0,0.2,0.8,100000,2.228691,0.386793,2.229085,0.386658,2.364724,0.382978,0.506752,0.237929,0.496806,0.830325,0.156175,0.362724
3,rewiring,er,250,0.1,0.0,0.2,1.2,100000,2.579699,0.445250,2.645225,0.471204,2.742667,0.555480,0.500606,0.249831,0.546255,1.008177,0.179297,0.440418
4,rewiring,er,250,0.1,0.0,0.2,1.6,100000,7.854710,0.402084,32.456240,5.061050,64.245012,10.079154,0.492880,0.277877,0.486494,2.220175,0.038018,0.969876


Unused functions

# def nclusters_meanshift(data, bw):
#     ops = {i: data[i] for i in range(len(data))}
#     sorted_ops = sorted(ops.items(), key = lambda kv:(kv[1], kv[0]))
#     A=np.array([el[1] for el in sorted_ops]).reshape(-1,1)
#     clustering = MeanShift(bandwidth=bw).fit(A)
#     lbls = clustering.labels_
#     labels = np.arange(len(sorted_ops))
#     for i in range(len(labels)):
#         cl = lbls[i]
#         labels[sorted_ops[i][0]]=cl
#     cluster_participation_dict = {}
#     for l in labels:
#         if l not in cluster_participation_dict:
#             cluster_participation_dict[l] = 1
#         else:
#             cluster_participation_dict[l] += 1
#     #computing effective number of clusters using function explained in the paper
#     C_num = 0
#     C_den = 0
#     for k in cluster_participation_dict:
#         C_num += cluster_participation_dict[k]
#         C_den += ((cluster_participation_dict[k])**2)
#     C_num = (C_num**2)
#     C = C_num/C_den
#     return C

In [ ]:
def steady_state_coevolving(model, name, max_iterations=100000, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=False):
    if not os.path.exists(f"snapshotGraphs/edgelist {name} 0.csv"):
        print(type(model.graph.graph))
        system_status = []
        steady_it = 0
        for it in tqdm.tqdm(range(0, max_iterations), disable=not progress_bar):            
            its = model.iteration(node_status)
            
            if it == 0:
                G = model.graph.graph
                nx.write_edgelist(G, f"snapshotGraphs/edgelist {name} {it}.csv", delimiter=",")
                with open(f"snapshotGraphs/opinions {name} {it}.txt", "w") as opfile:
                    first_values = list(its['status'].values())
                    for op in first_values:
                        opfile.write(str(op)+"\n")

            elif it > 0:
                old = np.array(list(system_status[-1]['status'].values()))
                actual = np.array(list(its['status'].values()))
                # res = np.abs(old - actual)
                if np.all((its['max_diff'] < sensibility)):
                    steady_it += 1
                else:
                    steady_it = 0

                if it % 500 == 0:
                    G = model.graph.graph
                    nx.write_edgelist(G, f"snapshotGraphs/edgelist {name} {it}.csv", delimiter=",")
                    with open(f"snapshotGraphs/opinions {name} {it}.txt", "w") as opfile:
                        for op in list(actual):
                            opfile.write(str(op)+"\n")

            system_status.append(its)
            
            if steady_it == nsteady:
                G = model.graph.graph
                nx.write_edgelist(G, f"snapshotGraphs/edgelist {name} {it}.csv", delimiter=",")
                with open(f"snapshotGraphs/opinions {name} {it}.txt", "w") as opfile:
                    for op in list(actual):
                        opfile.write(str(op)+"\n")
                        
                return system_status[:-nsteady]

        return system_status
# Save snapshots of graph during evolution
import sys
sys.path.append("/home/pansanella/mydata/GitHub/local_packages/")
sys.path.append("/data1/users/pansanella/mydata/GitHub/local_packages/")
sys.path.append("/data1/users/pansanella/mydata/GitHub/local_packages/netdspatch_local/")

import json
import networkx as nx
import ndlib_local.ndlib.models.ModelConfig as mc
import ndlib_local.ndlib.models.opinions as op
import warnings
warnings.filterwarnings("ignore")
n = 250
max_it = 100000
for graphname in ['er', 'ba']:
    if graphname == 'er':
        p = 0.1
        graph = nx.erdos_renyi_graph(n, p)
    else:
        p = 5
        graph = nx.barabasi_albert_graph(n, p)
    for pr in [0.5]:
        for e in [0.2, 0.3, 0.4]:
            for g in [0.0, 0.5, 1.0, 1.5]:
                name = f"rewiring {graphname}{p} pr{pr} e{e} g{g} mi{max_it}"
                model = op.AdaptiveAlgorithmicBiasModel(graph)
                config = mc.Configuration()
                config.add_model_parameter("epsilon", e)
                config.add_model_parameter("gamma", g)
                config.add_model_parameter("p", pr)
                model.set_initial_status(config)
                steady_status = steady_state_coevolving(model=model, name=name, max_iterations=max_it, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=True)                                